In [ ]:
import numpy as np
import pandas as pd
import random
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
import matplotlib.pyplot as plt


In [ ]:
rawdata = pd.read_csv('diabetes.csv')
print(rawdata.head())
df = rawdata[['diabetes']].copy()
df["diabetes"].replace(["No diabetes", "Diabetes"], [0,1] , inplace=True)
#print(df.head())
colsToNormalize = ["cholesterol", "glucose", "hdl_chol", "age", "height", "weight", "systolic_bp", "diastolic_bp", "waist", "hip"]
for i in range(0,len(colsToNormalize)):
    #print(df.loc[:,colsToNormalize[i]])
    df[colsToNormalize[i]] =(rawdata[colsToNormalize[i]] - rawdata[colsToNormalize[i]].mean())/rawdata[colsToNormalize[i]].std(ddof=False)

#df['gender'] = rawdata['gender']
dummies = pd.get_dummies(rawdata.gender)
#print(dummies)
df = pd.concat([df, dummies], axis = 'columns')

#splitData = rawdata['chol_hdl_ratio'].str.split(',', expand=True)
#df[chol_hdl_ratio] = splitData[0].astype(int)/splitData[1].astype(int)



In [ ]:
class NeuralNetwork:

    def __init__(self, numInputs, numHiddenNodes, learningRate, momentum):
        self.hiddenWeights = np.random.uniform(.1,.1,(numHiddenNodes,numInputs))
        self.outputWeights = np.random.uniform(.1,.1,(1, numHiddenNodes+1))
        self.numHiddenNodes = numHiddenNodes
        self.learningRate = learningRate
        self.momentum = momentum
        self.epochList = []
        self.expectedList = []
        self.predictedList = []
        self.previousOutputWeightDeltas = np.zeros((10,numHiddenNodes+1), dtype=float)
        self.previousHiddenWeightDeltas = np.zeros((numHiddenNodes, numInputs), dtype=float)
        self.debug = 0
        #self.p(self.weights)
    
    def train(self, data):
        return self.run_epoch(data, 1)
    
    def test(self, data):
        return self.run_epoch(data, 0)
    
    def sigmoid(self, x):
        y = 1/(1 + np.exp(-x))
        self.p("sigmoid for = ", y)
        return 1/(1 + np.exp(-x))
    
    def pp(self, label):
        self.p(label, "")
        
    def p(self, label, object):
        if self.debug == 1:
            print(label, object)
    
    def pr(self, label, object):
        print(label, object)
            
    def run_epoch(self, data, adjustWeights):
        #expectedList = np.array([.01,.99])
        #expectedList = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
        self.p("data=",data)
        self.p("self.outputWeights=",self.outputWeights)
        expectedList = []
        predictedList = []
        
        self.pp("##For each row of data")
        for i in range(0,data.shape[0]):
            self.p("row=",i)
            label = data[i,0].astype('int')
            #expectedOutputNodeList = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
            #set the bit for the label value to 1
            #expectedOutputNodeList[label] = 0.9
            expectedOutputNodeList = []
            expectedList.append(label)
            expectedOutputNodeList.append(label)
            self.p("expectedList=", expectedList)
            
            self.p("data[i]=",data[i])
            #input = data[i].astype('float16')/255
            input = data[i].astype('float16')
            #replace label column with 1 for bias
            #input[0] = 1
            self.p("input=",input)
       
            ###################
            #Calculate Forward
            ####################
            outputNodes = []
            hiddenNodes = []
            #append bias
            hiddenNodes.append(1)
            
            self.p("hiddenWeights=",self.hiddenWeights)
            self.p("outputWeights[t]=",self.outputWeights)
            
            #Inputs to Hidden
            self.pp("##inputs to hidden")
            loss = 0
            for h in range(self.numHiddenNodes):
                self.p("loop hidden node=",h)
                self.p("hiddenWeights=",self.hiddenWeights[h,:])
                hiddenNodes.append(self.sigmoid(np.inner(input, self.hiddenWeights[h,:])))
            
            self.p("hiddenNode values=",hiddenNodes)
            
            #Hidden to Outputs
            self.pp("##hidden to outputs")
            for t in range(0,self.outputWeights.shape[0]): #10):
                self.p("loop output node =",t)
                self.p("outputWeights[t]=",self.outputWeights[t])
                output = self.sigmoid(np.inner(hiddenNodes, self.outputWeights[t])) #self.weightsOuter[d,]))
                self.p("output=",output)
                outputNodes.append(output)
            
            predictedList.append(outputNodes[0])
            self.p("outputNodes=",outputNodes)
            self.p("predictedList=",predictedList)
            self.p("expectedOutputNodeList=",expectedOutputNodeList)
            
            ######################
            # Calculate Backwards
            #####################
           
            if adjustWeights == 1:
                self.pp("##Backwards")
                #Outputs to Hidden
                self.pp("##Output to hidden calculate error")
                outputError = []
                for t in range(len(outputNodes)):
                    self.p("loop output Node=",t)
                    self.p("outputNodes[t]=",outputNodes[t])
                    self.p("expectedOutputNodeList[t]=",expectedOutputNodeList[t])
                    outputError.append(outputNodes[t]*(1 - outputNodes[t])*(expectedOutputNodeList[t] - outputNodes[t])) 
                
                self.p("outputError=",outputError)   
    
                numOutputs = len(outputNodes)
                self.p("numOutputs=",numOutputs)
                self.p("outputError=",outputError)
                #outputError = np.divide(outputError, 2)            
                self.p("outputError=",outputError)   
                
                self.pp("##Hidden to inputs calculate error")
                hiddenError = []
                outputWeightsTransposed = self.outputWeights.transpose()
                self.p("outputWeightsTransposed=",outputWeightsTransposed)
                #skip first bias node, so start with 1
                for h in range(1,len(hiddenNodes)):
                    self.p("loop hidden node=",h)
                    self.p("hiddenNodes[r]=",hiddenNodes[h])
                    self.p("outputWeightsTransposed[h]=",outputWeightsTransposed[h])
                    hiddenError.append(hiddenNodes[h]*(1-hiddenNodes[h])*np.inner(outputWeightsTransposed[h],outputError))
                    self.p("hiddenError=",hiddenError[h-1])
                    
                self.p("hiddenError=",hiddenError)
                self.pp("## hidden to inputs")
                
                self.pp("##output to inner reset weights")
                outputWeightDeltas = []
                for t in range(self.outputWeights.shape[0]):
                    self.p("loop outer nodes =",t)
                    self.p("self.learningRate=",self.learningRate)
                    self.p("outputError[t]=", outputError[t])
                    self.p("hiddenNodes=",hiddenNodes)
                    self.p("np.dot=",np.dot(outputError[t], hiddenNodes))
                    self.p("self.learningRate*np.dot(outputError[t], hiddenNodes))=",self.learningRate*np.dot(outputError[t], hiddenNodes))
                    self.p("self.momentum=",self.momentum)
                    self.p("self.previousOutputWeightDeltas[t]=",self.previousOutputWeightDeltas[t])
                    self.p("self.momentum*self.previousOutputWeightDeltas[t]",self.momentum*self.previousOutputWeightDeltas[t])
                    outputWeightDeltas.append(self.learningRate*np.dot(outputError[t], hiddenNodes) + self.momentum*self.previousOutputWeightDeltas[t])
                    self.outputWeights[t] = self.outputWeights[t] + outputWeightDeltas[t] 
                    
                self.previousOutputWeightDeltas = outputWeightDeltas

                self.p("self.outputWeights=",self.outputWeights)
                
                hiddenWeightDeltas = []
                for h in range(self.hiddenWeights.shape[0]):
                    self.p("loop inner weight=",h)
                    hiddenWeightDeltas.append(self.learningRate*np.dot(hiddenError[h],input) + self.momentum * self.previousHiddenWeightDeltas[h])
                    self.hiddenWeights[h] = self.hiddenWeights[h] + hiddenWeightDeltas[h]
                    self.p("hidden weights h-", self.hiddenWeights[h])
                
                self.previousHiddenWeightDeltas = hiddenWeightDeltas
                self.p("self.hiddenWeights=",self.hiddenWeights)                

        self.storeLastEpoch(expectedList, predictedList)
        
        self.p("np.array(expectedList)=",np.array(expectedList))
        self.p("np.array(predictedList))=",np.array(predictedList))
        accuracy = (np.array(expectedList) == np.array(predictedList)).sum()/float(len(expectedList))*100 
        #accuracy = ()
        return outputError
    
    def storeLastEpoch(self, expectedList, predictedList):
        self.expectedList = expectedList
        self.predictedList = predictedList




In [ ]:

epochList = []
trainingAccuracyList = []
testingAccuracyList  =[]
expectedList = []
predictedList = []


results = []
inputResults = []

numHiddenNodes = 25
learningRate = .0001
momentum = 0.01
data_train = df_train
numInputs = data_train.shape[1]
network = NeuralNetwork(data_train.shape[1], numHiddenNodes, learningRate, momentum)
for e in range(100):
    np.random.shuffle(data_train)
    print("epoch:", e)
    train_accuracy = network.train(data_train)
    print("training accuracy:", train_accuracy)

#print("expectedList=",network.expectedList)
#print("predictedList=", network.predictedList)


#epochList.append(epoch)
#trainingAccuracyList.append(trainingAccuracy)
#testingAccuracyList.append(testingAccuracy)
#expectedList.append(perceptron.expectedList)
#predictedList.append(perceptron.predictionList)

In [ ]:
np.savetxt("results07102022-5.txt", results, delimiter=", ")
np.savetxt("array0601.txt", inputResults[0][0], delimiter=", ")
np.savetxt("array0602.txt", inputResults[0][1], delimiter=", ")
#np.savetxt("array0601.txt", inputResults[2][0], delimiter=", ")
#np.savetxt("array0602.txt", inputResults[2][1], delimiter=", ")

In [ ]:
plt.plot(epochList[0], trainingAccuracyList[0], label = "train")
plt.plot(epochList[0], testingAccuracyList[0], label = "test")
plt.legend()
plt.show

print(confusion_matrix(expectedList[0], predictedList[0]));

In [ ]:
plt.plot(epochList[1], trainingAccuracyList[1], label = "train")
plt.plot(epochList[1], testingAccuracyList[1], label = "test")
plt.legend()
plt.show

print(confusion_matrix(expectedList[1], predictedList[1]));

In [ ]:
plt.plot(epochList[2], trainingAccuracyList[2], label = "train")
plt.plot(epochList[2], testingAccuracyList[2], label = "test")
plt.legend()
plt.show

print(confusion_matrix(expectedList[2], predictedList[2]));